# Redes Neuronales

## Introducción

Las redes neuronales artificiales son modelos que aprenden funciones complejas a partir de datos. En este práctico, trabajaremos con un problema de **regresión**: predecir el precio de viviendas a partir de características como número de habitaciones, edad, etc.

Exploraremos distintas arquitecturas de redes neuronales y analizaremos cómo afecta su complejidad al desempeño del modelo.


## Dependencias

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Carga de datos

In [ ]:
# Cargar el dataset de ejemplo que está en Keras
from tensorflow.keras.datasets import boston_housing

(X_train, y_train), (X_val, y_val) = boston_housing.load_data()

### Shapes

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Arquitectura de la Red Neuronal
El modelo que estamos utilizando es una red neuronal **secuencial** de tipo **feedforward**, diseñada para resolver un problema de **regresión** (predecir el precio de una casa).


In [ ]:
model_1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Capa oculta con la entrada siendo la cantidad de features
    layers.Dense(32, activation='relu'),  # Capa oculta
    layers.Dense(1)  # Capa de salida (el precio a predecir)
])

- `Dense(64, activation='relu')`: primera capa oculta, con 64 neuronas. Utiliza la función ReLU para introducir no linealidad. Esta capa recibe como entrada un vector de 13 características (una por atributo del dataset).
  
- `Dense(32, activation='relu')`: segunda capa oculta, con menor cantidad de neuronas. Se busca una representación más compacta de los datos.

- `Dense(1)`: capa de salida. Produce un único valor continuo correspondiente al precio predicho.

In [ ]:
model_1.summary()

### 🔍 ¿Por qué esta arquitectura?
- Las redes densas son adecuadas cuando no hay estructura espacial en los datos (como ocurre con imágenes).
- La función `ReLU` es rápida de computar y evita problemas de desvanecimiento del gradiente.
- La capa de salida tiene una sola neurona porque estamos prediciendo un único valor numérico (no una clase).

## 🔧 Compilación del Modelo

La compilación del modelo especifica **cómo será entrenado**. En este caso, usamos:

- **Optimizador: `adam`**  
  Un optimizador eficiente que ajusta automáticamente la tasa de aprendizaje de cada peso.

- **Función de pérdida: `mean_squared_error` (MSE)**  
  Es la función que el modelo intenta minimizar. Penaliza más los errores grandes, lo que puede ayudar a evitar grandes desviaciones en las predicciones.

- **Métrica: `mae` (Mean Absolute Error)**  
  Métrica adicional que se reporta durante el entrenamiento. Es útil porque está en las mismas unidades que el objetivo (precio de las casas), y es fácil de interpretar.

> 💡 El modelo entrenará para minimizar MSE, pero durante el proceso también se mostrará el MAE como referencia de desempeño.


In [ ]:
model_1.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

## 🚀 Entrenamiento del Modelo

Utilizamos el método `fit()` para entrenar la red neuronal. Este método ajusta los pesos del modelo para minimizar la función de pérdida.

### Parámetros clave:

- **`X_train`, `y_train`**: conjunto de entrenamiento (entradas y salidas).
- **`validation_data=(X_val, y_val)`**: conjunto de validación que permite evaluar el modelo en datos no vistos, detectando sobreajuste.
- **`epochs=100`**: número de veces que el modelo verá todo el conjunto de entrenamiento.
- **`batch_size=32`**: tamaño de los bloques de datos usados para actualizar los pesos.

Al final, la variable `history` almacena el historial del entrenamiento, incluyendo la pérdida y las métricas por época. Esto nos permite visualizar cómo evoluciona el modelo con el tiempo.


In [ ]:
history_1 = model_1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

## Evaluacion

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_mae = history.history['mae']
    val_mae = history.history['val_mae']

    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(8, 4))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'b-', label='Pérdida en entrenamiento')
    plt.plot(epochs, val_loss, 'r-', label='Pérdida en validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida (Loss)')
    plt.title('Curva de pérdida en entrenamiento y validación')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_mae, 'orange', label='MAE en entrenamiento')
    plt.plot(epochs, val_mae, 'green', label='MAE en validación')
    plt.xlabel('Épocas')
    plt.ylabel('MAE (Error Absoluto Medio)')
    plt.title('Curva de MAE en entrenamiento y validación')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_training_history(history_1)

- Si las curvas de entrenamiento y validación bajan y se estabilizan juntas → el modelo aprende correctamente.
- Si la pérdida de validación empieza a subir mientras la de entrenamiento sigue bajando → el modelo está **sobreajustando** (*overfitting*).
- Si ambas se mantienen altas → el modelo **no está aprendiendo bien** (*underfitting*).

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
y_pred_1 = model_1.predict(X_val)

In [ ]:
mae_1 = mean_absolute_error(y_val, y_pred_1)
mse_1 = mean_squared_error(y_val, y_pred_1)
print(f"MAE: {mae_1}")
print(f"MSE: {mse_1}")

## Segundo modelo

In [ ]:
model_2 = keras.Sequential([
    layers.Dense(500, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(200, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

In [ ]:
model_2.summary()

- `Dense(500, activation='relu')`: capa muy ancha que permite capturar combinaciones complejas entre las entradas.
- Capas subsiguientes: tamaños decrecientes (200, 128, 64, 32), siguiendo una forma de "embudo".
- Cada capa aplica una función `ReLU`, que introduce no linealidad y permite aprender funciones más complejas.
- La salida (`Dense(1)`) es un único valor continuo, ideal para regresión.

In [ ]:
model_2.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
history_2 = model_2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, batch_size=32, verbose=False)


- **`epochs=500`**: permite que la red tenga más tiempo para ajustar sus pesos y aprender patrones complejos. Sin embargo, debemos tener cuidado con el **sobreajuste (overfitting)**.
- **`verbose=False`**: desactiva la salida en consola para no saturar el notebook con 500 líneas. Para evaluar el progreso, es importante **graficar la evolución de la pérdida y del MAE** usando el historial (`history`).

> 💡 El aumento de épocas debe ir acompañado de una buena estrategia de evaluación para saber cuándo el modelo deja de mejorar.

In [ ]:
plot_training_history(history_2)

In [ ]:
y_pred_2 = model_2.predict(X_val)
mae_2 = mean_absolute_error(y_val, y_pred_2)
mse_2 = mean_squared_error(y_val, y_pred_2)
print(f"MAE: {mae_2}")
print(f"MSE: {mse_2}")

## Tercer modelo (con regularización)

Este modelo mantiene la misma arquitectura profunda del segundo modelo, pero **agrega una capa de regularización** para mejorar su capacidad de generalización.

### 🔧 ¿Qué cambia?

- Se añade `Dropout(0.5)` antes de la capa de salida. Esta capa "apaga" aleatoriamente el 50% de las neuronas durante el entrenamiento.
- Esto fuerza al modelo a **no depender de caminos específicos** y lo hace más robusto frente a nuevos datos.

### 🎯 ¿Por qué usar Dropout?

- Ayuda a prevenir el **sobreajuste (overfitting)**, especialmente en redes con muchas capas y neuronas.
- Mejora la capacidad del modelo de **generalizar**.
- Es una de las formas más comunes y efectivas de **regularización en redes neuronales**.

> 💡 Dropout solo está activo durante el entrenamiento. En predicción, se desactiva automáticamente.

In [ ]:
model_3 = keras.Sequential([
    layers.Dense(500, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(200, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

In [ ]:
model_3.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


### Early Stopping

Entrenar una red neuronal por muchas épocas puede llevar al **sobreajuste**, donde el modelo aprende demasiado bien los datos de entrenamiento pero pierde capacidad de generalización.

Para evitarlo, utilizamos `EarlyStopping`, una técnica que:

- **Monitorea el desempeño del modelo en el conjunto de validación** (`val_loss`).
- **Detiene automáticamente el entrenamiento** si no hay mejora tras un número definido de épocas (en este caso, 30).
- **Restaura los pesos óptimos** alcanzados durante el entrenamiento (`restore_best_weights=True`).

### 🔧 Ventajas:
- Ahorra tiempo.
- Evita el sobreajuste.
- Mantiene el mejor modelo encontrado automáticamente.

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)

history_3 = model_3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, batch_size=32, callbacks=[early_stopping], verbose=True)

In [ ]:
plot_training_history(history_3)

In [ ]:
y_pred_3 = model_3.predict(X_val)
mae_3 = mean_absolute_error(y_val, y_pred_3)
mse_3 = mean_squared_error(y_val, y_pred_3)
print(f"MAE: {mae_3}")
print(f"MSE: {mse_3}")

## Resumen Métricas

In [ ]:
#create dataframe of the mae and mse
df_metrics = pd.DataFrame({'Modelo': ['Modelo 1', 'Modelo 2', 'Modelo 3'], 'MAE': [mae_1, mae_2, mae_3], 'MSE': [mse_1, mse_2, mse_3]})
df_metrics
